In [ ]:
import datacube
from planetary_computer import sign_url
import holoviews as hv

from dea_tools.plotting import rgb

import hvplot.xarray
hv.extension('bokeh')

In [ ]:
dc = datacube.Datacube()

In [ ]:
# Find data over Fiji
lon = [177, 179]
lat = [-18.5, -17]
datasets = dc.find_datasets(product="sentinel_1_rtc", lon=lon, lat=lat)

print(f"Found {len(datasets)} datasets")

In [ ]:
data = dc.load(datasets=datasets[0:1], output_crs="EPSG:3832", resolution=(-25, 25), patch_url=sign_url, lon=lon, lat=lat)
data = data.where(data.vv != -32768)

data

In [ ]:
# Remove the time dimension
data = data.squeeze("time")

In [ ]:
window_size = 3

# Add vv/vh
data["vv_vh"] = data.vv / data.vh

# Smooth the dataset with a rolling mean
data["vv_smoothed"] = data.vv.rolling(x=window_size, y=window_size, center=True).mean()
data["vh_smoothed"] = data.vh.rolling(x=window_size, y=window_size, center=True).mean()
data["vv_vh_smoothed"] = data.vv_vh.rolling(x=window_size, y=window_size, center=True).mean()

median = data[["vv_smoothed", "vh_smoothed", "vv_vh_smoothed"]].median()

In [ ]:
rgb(data[["vv_smoothed", "vh_smoothed", "vv_vh_smoothed"]] / median, bands=["vv_smoothed", "vh_smoothed", "vv_vh_smoothed"], size=10)

In [ ]:
data.vv_smoothed.hvplot.hist(bins=100, bin_range=(0, 0.5), title="VV Histogram")

In [ ]:
data["water"] = data.vv_smoothed < 0.04
data["water"] = data.water.where(data.water)

In [ ]:
import folium
import odc.geo.xr

m = folium.Map(control_scale=True, tiles=None)

data.vv_smoothed.odc.add_to(m, opacity=1.0, name="vv_smoothed", vmin=0, vmax=0.5)
data.vv.odc.add_to(m, opacity=1.0, name="vv", vmin=0, vmax=0.5)
data.water.odc.add_to(m, opacity=1.0, name="water", vmin=0, vmax=1, cmap="Blues")

# Zoom map
m.fit_bounds(data.odc.map_bounds())

tile = folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr = 'Esri',
    name = 'Esri Satellite',
    overlay = False,
    control = True
).add_to(m)
folium.TileLayer('openstreetmap').add_to(m)

folium.LayerControl().add_to(m)
display(m)